# Распознавание ручек/карандашей

In [1]:
from ultralytics import YOLO
from pathlib import Path
import gc
import torch

In [2]:
DATA_ROOT = Path(r"datasets\testcase_pen")
PROJECT_NAME = "pen_penclin_detection"

DATA_ROOT2 = Path(r"datasets\testcase_pen2")

## Обучим первую версию модели ##

In [3]:
def main():
    
    model = YOLO("yolo12s.pt")
    
    model.train(
        data=DATA_ROOT / "data.yaml",
        epochs=50,
        imgsz=640,
        batch=0.7,
        device=0,
        project=PROJECT_NAME,
    )

if __name__ == "__main__":
    main()


New https://pypi.org/project/ultralytics/8.3.233 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.7, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets\testcase_pen\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=Fal

C:\Users\mrsli\anaconda3\envs\work_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 70.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4070) 11.99G total, 0.13G reserved, 0.10G allocated, 11.76G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     9253910       21.52         0.984            50          2724        (1, 3, 640, 640)                    list
     9253910       43.05         1.447           177          3354        (2, 3, 640, 640)                    list
     9253910        86.1         2.259           158          3452        (4, 3, 640, 640)                    list
     9253910       172.2         3.890            83          3538        (8, 3, 640, 640)                    list
     9253910       3

### СЧИТАЕМ МЕТРИКИ НА ВАЛИДАЦИОННОМ НАБОРЕ ###

In [5]:
model = YOLO(r"pen_penclin_detection\train\weights\best.pt")

metrics = model.val(data=DATA_ROOT / "data.yaml", device=0, batch=1, imgsz=640)
# усреднённые по классам метрики 
P = float(metrics.box.mp)
R = float(metrics.box.mr)
F1 = 2 * P * R / (P + R + 1e-16)

print(f"Precision: {P:.4f}")
print(f"Recall:    {R:.4f}")
print(f"F1-score:  {F1:.4f}")

Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv12s summary (fused): 159 layers, 9,231,654 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2622.71369.6 MB/s, size: 215.1 KB)
val: Scanning C:\Anaconda_notebooks\datasets\testcase_pen\valid\labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10/10 10.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 5.4it/s 1.8s0.2s
                   all         10         44      0.996      0.985      0.995      0.869
                   pen         10         26          1      0.971      0.995      0.903
                pencil         10         18      0.993          1      0.995      0.835
Speed: 2.4ms preprocess, 35.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to C:\Anaconda_notebooks\runs\detect\val48
Precision: 0.9964
Recall:    0.9853
F1-sc

### ЗАПУСК ИНФЕРЕНСА НА ВИДЕО ###

In [6]:
model = YOLO(r"pen_penclin_detection\train\weights\best.pt")
result = model.predict(
    source=DATA_ROOT / "test.MOV",
    conf=0.5,
    device=0,
    save=True,
    save_conf=True,
    vid_stride=2,
    project=PROJECT_NAME,   # папка с результатами
    name = 'test_vid',
    exist_ok=True,     # не ругаться, если папка уже есть
)



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/971) C:\Anaconda_notebooks\datasets\testcase_pen\test.MOV: 640x384 4 pens, 1 pencil, 57.0ms
video 1/1 (frame 2/971) C:\Anaconda_notebooks\datasets\testcase_pen\test.MOV: 640x384 4 pens, 1 pencil, 47.1ms
video 1/1 (frame 3/971) C:\Anaconda_notebooks\datasets\testcase_pen\test.MOV: 640x384 4 pens, 1 pencil, 71.0ms
video 1/1 (frame 4/971) C:\Anaconda_notebooks\datasets\testcase_pen\test.MOV: 640x384 4 pens, 1 pencil, 27.5ms
video 1/1 (fram

## Рассмотрим на примере какой conf у каждого предсказания ДО работы NMS ##

In [8]:
model = YOLO(r"pen_penclin_detection\train\weights\best.pt")

def inspect_frame():
    results = model.predict(
        source=DATA_ROOT / "neg_case2.png",
        project=PROJECT_NAME,
        name = "negative_examples",
        conf=0.1,
        iou=0.99,     # почти отключаем NMS
        verbose=False,
        save = True,
        
    )

    r = results[0]
    boxes = r.boxes

    cls_ids = boxes.cls.cpu().numpy()
    confs = boxes.conf.cpu().numpy()
    xyxy = boxes.xyxy.cpu().numpy()

    for (x1, y1, x2, y2), cls_id, conf in zip(xyxy, cls_ids, confs):
        name = model.names[int(cls_id)]
        print(
            f"{name:6s} conf={conf:.3f} "
            f"box=({x1:.1f}, {y1:.1f}, {x2:.1f}, {y2:.1f})"
        )

if __name__ == "__main__":
    inspect_frame()


Results saved to C:\Anaconda_notebooks\pen_penclin_detection\negative_examples2
pen    conf=0.923 box=(323.9, 307.8, 671.9, 705.7)
pen    conf=0.918 box=(323.3, 306.2, 675.3, 710.1)
pencil conf=0.908 box=(612.9, 117.9, 825.6, 665.4)
pen    conf=0.908 box=(325.4, 305.4, 671.2, 707.7)
pen    conf=0.907 box=(323.8, 307.6, 678.3, 706.2)
pen    conf=0.886 box=(325.1, 304.9, 670.5, 710.8)
pencil conf=0.880 box=(617.1, 118.9, 827.8, 667.4)
pen    conf=0.872 box=(325.5, 307.5, 672.1, 709.7)
pen    conf=0.867 box=(467.5, 1094.5, 1008.8, 1383.5)
pen    conf=0.859 box=(323.4, 307.6, 677.6, 709.2)
pencil conf=0.847 box=(618.7, 120.6, 833.2, 666.4)
pencil conf=0.846 box=(616.6, 118.6, 826.2, 665.8)
pen    conf=0.840 box=(468.4, 1091.1, 1010.0, 1383.8)
pencil conf=0.835 box=(612.4, 119.3, 827.2, 668.4)
pencil conf=0.833 box=(616.5, 119.0, 829.2, 668.9)
pen    conf=0.828 box=(468.0, 1088.4, 1011.9, 1385.2)
pen    conf=0.823 box=(466.1, 1093.6, 1017.6, 1386.0)
pen    conf=0.819 box=(464.4, 1096.6, 101

## Обучаем новую модель после осознания проблемы - повороты ##

In [9]:
model = YOLO("yolo12s.pt")

model.train(
    data=DATA_ROOT2 / "data.yaml",
    epochs=50,
    imgsz=640,
    batch=0.7,
    device=0,
    project=PROJECT_NAME,
    name="yolo12s_augmented",
    # усиливаем аугментации
    degrees=30,     # случайный поворот ±30°
    scale=0.5,      # масштабирование
    shear=10.0,     # сдвиг/срез
    fliplr=0.5,     # горизонтальные флипы
    flipud=0.0,
)

New https://pypi.org/project/ultralytics/8.3.233 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.7, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets\testcase_pen2\data.yaml, degrees=30, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo12s_augmented, nbs

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000246322CD820>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

## Проверим работу новой модели на проблемном видео ##

In [10]:
model = YOLO(r"pen_penclin_detection\yolo12s_augmented\weights\best.pt")

metrics = model.val(data=DATA_ROOT2 / "data.yaml", device=0, batch=1, imgsz=640)
# усреднённые по классам метрики 
P = float(metrics.box.mp)
R = float(metrics.box.mr)
F1 = 2 * P * R / (P + R + 1e-16)

print(f"Precision: {P:.4f}")
print(f"Recall:    {R:.4f}")
print(f"F1-score:  {F1:.4f}")


result = model.predict(
    source=DATA_ROOT / "test.MOV",
    conf=0.5,
    device=0,
    save=True,
    save_conf=True,
    vid_stride=2,
    project=PROJECT_NAME,   # папка с результатами
    exist_ok=True,     # не ругаться, если папка уже есть
)


Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv12s summary (fused): 159 layers, 9,231,654 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2756.2235.4 MB/s, size: 218.4 KB)
val: Scanning C:\Anaconda_notebooks\datasets\testcase_pen2\valid\labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10/10  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 5.4it/s 1.9s0.1s
                   all         10         44        0.9      0.889       0.93      0.602
                   pen         10         26       0.98          1      0.995      0.671
                pencil         10         18       0.82      0.778      0.865      0.533
Speed: 1.0ms preprocess, 26.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\Anaconda_notebooks\runs\detect\val49
Precision: 0.9004
Recall:    0.8889
F1-score:  0.8

## А теперь протестируем разные модели ##

In [11]:
def gpu_cleanup(sync: bool = False):
    """Чистим кеш CUDA"""
    gc.collect()
    if torch.cuda.is_available():
        if sync:
            torch.cuda.synchronize()
        torch.cuda.empty_cache()
        # иногда полезно, но не у всех сборок CUDA есть:
        try:
            torch.cuda.ipc_collect()
        except Exception:
            pass

In [12]:
# куда сохранять видео с боксами
INFER_PROJECT = "pen_pencil_inference"
TRAIN_PROJECT = "models_compare"

# список моделей
MODELS = ["yolo12s",
          "yolo12n",
          "yolo12m",
          "yolo11s",
          "rtdetr-l"
]


def main():
    results = []

    for model_name in MODELS:
        print(f"\n=== Оцениваем модель: {model_name} ===\n")

        # --- загрузка модели ---
        model = YOLO(str(model_name) + ".pt")

        # --- обучение модели ---

        model.train(
            data=DATA_ROOT2 / "data.yaml",
            epochs=50,
            imgsz=640,
            batch=0.7,
            device=0,
            project=TRAIN_PROJECT,
            name=model_name,
            exist_ok=True,   # чтобы можно было перезапустить без ошибки
    
            # аугментации
            degrees=30.0,    # случайный поворот ±30°
            scale=0.5,       # масштабирование
            shear=10.0,      # сдвиг
            fliplr=0.5,      # горизонтальные флипы
            flipud=0.0,
        )

        model = YOLO(r"models_compare\\" + model_name + "\weights\\best.pt")
        
        # --- валидация ---
        metrics = model.val(
            data=DATA_ROOT2 / "data.yaml",
            device=0,
            batch=1,
            imgsz=640,
            verbose=False,
        )

        # усреднённые по классам метрики
        P = float(metrics.box.mp)
        R = float(metrics.box.mr)
        F1 = 2 * P * R / (P + R + 1e-16)
        mAP50 = float(metrics.box.map50)
        mAP5095 = float(metrics.box.map)

        results.append({
            "model": model_name,
            "precision": P,
            "recall": R,
            "F1": F1,
            "mAP50": mAP50,
            "mAP50-95": mAP5095,
        })

        print(f"Val metrics for {model_name}:")
        print(f"  Precision: {P:.4f}")
        print(f"  Recall:    {R:.4f}")
        print(f"  F1-score:  {F1:.4f}")
        print(f"  mAP@0.5:   {mAP50:.4f}")
        print(f"  mAP@0.5:0.95: {mAP5095:.4f}")

        # --- инференс на тестовом видео ---
        print(f"\nЗапускаем инференс на тестовом видео для {model_name}...\n")
        model.predict(
            source=DATA_ROOT / "test.MOV",
            conf=0.5,
            iou=0.45,
            device=0,
            save=True,
            save_conf=True,
            project=INFER_PROJECT,
            name=model_name,   # у каждой модели своя подпапка
            exist_ok=True,     # не создавать model_name2 и т.п.
            vid_stride=2,      # каждый 2 кадр
            show=False,
        )

        gpu_cleanup()

    # --- печать общей таблицы по всем моделям ---
    print("\n=== Сводная таблица по моделям ===\n")

    header = f"{'Model':15s} {'P':>8s} {'R':>8s} {'F1':>8s} {'mAP50':>8s} {'mAP50-95':>10s}"
    print(header)
    print("-" * len(header))

    for r in results:
        print(
            f"{r['model']:15s} "
            f"{r['precision']:8.4f} "
            f"{r['recall']:8.4f} "
            f"{r['F1']:8.4f} "
            f"{r['mAP50']:8.4f} "
            f"{r['mAP50-95']:10.4f}"
        )

    print("\nВидео с детекцией лежат в папке:", INFER_PROJECT)


if __name__ == "__main__":
    main()


<>:43: SyntaxWarning: invalid escape sequence '\w'
<>:43: SyntaxWarning: invalid escape sequence '\w'
C:\Users\mrsli\AppData\Local\Temp\ipykernel_18580\2943060350.py:43: SyntaxWarning: invalid escape sequence '\w'
  model = YOLO(r"models_compare\\" + model_name + "\weights\\best.pt")



=== Оцениваем модель: yolo12s ===

New https://pypi.org/project/ultralytics/8.3.233 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.7, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets\testcase_pen2\data.yaml, degrees=30.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=1.0, multi_sca

C:\Users\mrsli\anaconda3\envs\work_env\Lib\site-packages\torch\autograd\graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/50      2.68G      2.182      1.752       2.82         17        640: 100% ━━━━━━━━━━━━ 20/20 1.9it/s 10.6s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 2.3it/s 0.9s2.7s
                   all         10         44     0.0894     0.0833     0.0203     0.0153

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       2/50      2.91G      2.169      1.076      3.643         15        640: 0% ──────────── 0/20  0.2s

C:\Users\mrsli\anaconda3\envs\work_env\Lib\site-packages\torch\autograd\graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

      48/50      3.62G     0.4053      0.461     0.3129         14        640: 100% ━━━━━━━━━━━━ 20/20 4.5it/s 4.5s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 14.3it/s 0.1s
                   all         10         44      0.941      0.934      0.946      0.507

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      49/50      3.62G     0.3395     0.4002     0.2127         12        640: 0% ──────────── 0/20  0.3s

C:\Users\mrsli\anaconda3\envs\work_env\Lib\site-packages\torch\autograd\graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      49/50      3.62G     0.3906     0.4475     0.3272         14        640: 100% ━━━━━━━━━━━━ 20/20 4.2it/s 4.8s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 14.3it/s 0.1s
                   all         10         44      0.975      0.952      0.986      0.501

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      50/50      3.62G     0.3991     0.5125     0.3078          9        640: 5% ╸─────────── 1/20 1.6it/s 0.4s<11.7s

C:\Users\mrsli\anaconda3\envs\work_env\Lib\site-packages\torch\autograd\graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      50/50      3.62G     0.4215     0.4579     0.3282         14        640: 100% ━━━━━━━━━━━━ 20/20 4.5it/s 4.5s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 15.9it/s 0.1s
                   all         10         44      0.946      0.956      0.985      0.482

50 epochs completed in 0.084 hours.
Optimizer stripped from C:\Anaconda_notebooks\models_compare\rtdetr-l\weights\last.pt, 66.1MB
Optimizer stripped from C:\Anaconda_notebooks\models_compare\rtdetr-l\weights\best.pt, 66.1MB

Validating C:\Anaconda_notebooks\models_compare\rtdetr-l\weights\best.pt...
Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
rt-detr-l summary: 302 layers, 31,987,850 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 7.0it/s 0.3s0.6s
                   all         10         44      0.966  

# Продолжим обучать ЛУЧШУЮ модель, но с меньшим lr - может сможем еще чуть-чуть повысить качество

In [13]:
BASE_RUN = Path('models_compare') / "yolo12s"
PRETRAINED_WEIGHTS = BASE_RUN / "weights" / "best.pt"

def main():
    print(f"Используем веса: {PRETRAINED_WEIGHTS}")

    model = YOLO(str(PRETRAINED_WEIGHTS))

    model.train(
        data=DATA_ROOT2 / "data.yaml",
        epochs=200,
        imgsz=640,
        batch=0.7,
        device=0,
        project=TRAIN_PROJECT,
        name="yolo12s_long_ft",
        exist_ok=True,

        # воспользуемся затуханием lr
        optimizer="AdamW",
        lr0=1e-4,           # стартовый lr
        lrf=0.005, 
        cos_lr=True,
        warmup_epochs=5.0,   #3 -> 5 (зачем? вопрос. но по логике на маленьком датасете мы при дообучении можем убить начальные веса)
        patience=30,         # электричество тоже денег стоит

        # аугментации
        degrees=30.0,
        scale=0.5,
        shear=10.0,
        fliplr=0.5,
        flipud=0.0,

        # можем себе позволить, вдруг качество чуть выше будет
        amp = False,
    )

if __name__ == "__main__":
    main()


Используем веса: models_compare\yolo12s\weights\best.pt
New https://pypi.org/project/ultralytics/8.3.233 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=0.7, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=datasets\testcase_pen2\data.yaml, degrees=30.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.005, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=models_compare\yolo12s\we

# Получим метрики валидации + прогоним тест на лучшей модели

In [15]:
model = YOLO(r"models_compare\yolo12s_long_ft\weights\best.pt")

metrics = model.val(data=DATA_ROOT2 / "data.yaml", device=0, batch=1, imgsz=640)
# усреднённые по классам метрики 
P = float(metrics.box.mp)
R = float(metrics.box.mr)
F1 = 2 * P * R / (P + R + 1e-16)

print(f"Precision: {P:.4f}")
print(f"Recall:    {R:.4f}")
print(f"F1-score:  {F1:.4f}")


result = model.predict(
    source=DATA_ROOT / "test.MOV",
    conf=0.5,
    device=0,
    save=True,
    save_conf=True,
    vid_stride=2,
    project=PROJECT_NAME,   # папка с результатами
    exist_ok=True,     # не ругаться, если папка уже есть
)


Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv12s summary (fused): 159 layers, 9,231,654 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2056.5277.3 MB/s, size: 216.5 KB)
val: Scanning C:\Anaconda_notebooks\datasets\testcase_pen2\valid\labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10/10  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 4.6it/s 2.2s0.1s
                   all         10         44      0.995          1      0.995       0.78
                   pen         10         26      0.991          1      0.995      0.834
                pencil         10         18          1          1      0.995      0.727
Speed: 1.8ms preprocess, 37.8ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to C:\Anaconda_notebooks\runs\detect\val56
Precision: 0.9952
Recall:    1.0000
F1-score:  0.9